In [2]:
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from keras.utils import to_categorical
from ml_utils import grab_labels, feature_from_file, translate_bytes, feature_from_list, convert_multiclass_labels
import csv

le = LabelEncoder()
naivetruth_path = "/Users/ryan/Documents/CS/CDAC/xtract_autoencoder/automated_training_results/matio_results/nist_dataset.csv"

labels, file_paths = grab_labels(naivetruth_path)
labels.pop(0) #Gets rid of headers
file_paths.pop(0)

y = to_categorical(le.fit_transform(labels), 7).tolist()
print('here')
file_paths, y = convert_multiclass_labels(file_paths, y)

print('1')
x = feature_from_list(file_paths)
print('2')
x = translate_bytes(x) / 255

print('3')
x_train, x_test, y_train, y_test = train_test_split(x, y)

print(x_train)
print('4')

here
88334
1
2
3
[[0.19215686 0.2        0.03921569 ... 0.18823529 0.2        0.1254902 ]
 [0.30196078 0.40392157 0.22352941 ... 0.19607843 0.20392157 0.22352941]
 [0.39215686 0.41176471 0.44705882 ... 0.         0.         0.        ]
 ...
 [0.         0.         0.         ... 0.         0.         0.        ]
 [0.19215686 0.20392157 0.03921569 ... 0.2        0.2        0.20392157]
 [0.23529412 0.24705882 0.47058824 ... 0.28627451 0.3254902  0.31372549]]
4


In [6]:
print(le.classes_)
for unique in set(labels):
    print("{} is {} and there are {} files".format(unique, ((labels.count(unique) / len(labels)) * 100), labels.count(unique)))

['ase' 'crystal' 'csv' 'dft' 'em' 'image' 'none']
ase is 14.173510233954007 and there are 98186 files
em is 0.08300336488423557 and there are 575 files
csv is 0.08906621936273625 and there are 617 files
dft is 2.7737559239140634 and there are 19215 files
none is 70.01889589645799 and there are 485051 files
image is 0.16470754666593526 and there are 1141 files
crystal is 12.69706081476103 and there are 87958 files


In [8]:
from keras.models import Sequential
from keras.layers import Dense

classifier_model = Sequential()
classifier_model.add(Dense(256, activation='relu', input_shape=(512,)))
classifier_model.add(Dense(128, activation='relu'))
classifier_model.add(Dense(64, activation='relu'))
classifier_model.add(Dense(32, activation='relu'))
classifier_model.add(Dense(7, activation='sigmoid'))

classifier_model.compile(optimizer='sgd', loss='binary_crossentropy', metrics=['accuracy'])
classifier_model.summary()

history = classifier_model.fit(x_train, y_train,
                               epochs=20,
                               batch_size = 16,
                               shuffle=True,
                               validation_data=(x_test, y_test)) 

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 256)               131328    
_________________________________________________________________
dense_2 (Dense)              (None, 128)               32896     
_________________________________________________________________
dense_3 (Dense)              (None, 64)                8256      
_________________________________________________________________
dense_4 (Dense)              (None, 32)                2080      
_________________________________________________________________
dense_5 (Dense)              (None, 7)                 231       
Total params: 174,791
Trainable params: 174,791
Non-trainable params: 0
_________________________________________________________________
Train on 519557 samples, validate on 173186 samples
Epoch 1/20
519557/519557 [==============================] - 49

KeyboardInterrupt: 

In [ ]:
def convert_result_to_index(result):
    for 